
<div style="background-color: black; color: white; padding: 6px; border-radius: 8px;font-size: 1.5em;">
<img src="Title2.png" alt="" style="height: 10em; margin-right: 10px;">
</div>


### Abstract
In this lab, you will learn about **Quantum Error Suppression and Error Mitigation (QESEM)** through hands-on experiments.  
With QESEM, users can run quantum circuits on noisy QPUs and obtain highly accurate, error-free results with minimal QPU time overhead, close to theoretical limits.  
When executing a circuit, QESEM runs a device characterization protocol tailored to your circuit, yielding a reliable noise model for the errors occurring in the circuit. Based on the characterization, QESEM first implements noise-aware transpilation to map the input circuit onto a set of physical qubits and gates, which minimizes the noise affecting the target observable. These include the natively available gates (CX/CZ on IBM® devices), as well as additional gates optimized by QESEM, forming QESEM’s extended gate set. QESEM then runs a set of characterization-based error suppression (ES) and error mitigation (EM) circuits on the QPU and collects their measurement outcomes. These are then classically post-processed to provide an unbiased expectation value and an error bar for each observable, corresponding to the requested accuracy.

<div>
<img src="qesem_workflow.svg" width="800"/>
</div>

## Setup

In [ ]:
# Install dependencies
%pip install "qiskit[visualization]>=2.0.0" "qiskit-ibm-runtime>=0.40.0" "qiskit-ibm-catalog>=0.8.0" "qiskit-aer>=0.17.1" "jupyter>=1.1.1" "networkx>=3.5" "tqdm>=4.67.1" "scipy"

In [ ]:
# Imports

%matplotlib inline   

import qiskit
from qiskit.quantum_info import Pauli, SparsePauliOp, Statevector
from qiskit_ibm_runtime.fake_provider import FakeFez
from qiskit.visualization import circuit_drawer
from qiskit import transpile, QuantumCircuit
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_catalog import QiskitFunctionsCatalog
from qiskit_ibm_runtime import QiskitRuntimeService

import matplotlib.pyplot as plt
import pylatexenc
import networkx as nx
from scipy.optimize import curve_fit
from matplotlib.ticker import MaxNLocator
from tqdm import tqdm
import numpy as np
import json

import time
from datetime import datetime
from IPython.display import display, Math

from grader import *

<div class="alert alert-block alert-warning">

**Exclusive Access to Qiskit Functions**

As part of Qiskit Global Summer School (QGSS), participants with a Premium or Flex Plan have limited-time trial access to Qiskit Functions. Access is exclusive and subject to your organization’s administrator approval. Complete [this form](https://airtable.com/appj8IrSNZGz4l4BB/pag8WgWdUr5uSJGZA/form) to request access.

If you encounter the error `QiskitServerlessException: Credentials couldn't be verified`. in the cell below, it means your access to Qiskit Functions is not yet active. Please check back later after your request has been processed.

**Note: Running this lab will consume QPU time from your organization’s account. Estimated QPU usage is provided before each cell that executes on a QPU. Please monitor your usage and consult your organization admin if you’re unsure about your allocated QPU time for QGSS Functions labs.**

</div>

In [ ]:
# Load the Qiskit Functions Catalog
your_api_key = "deleteThisAndPasteYourAPIKeyHere"
your_crn = "deleteThisAndPasteYourCRNHere"

catalog = QiskitFunctionsCatalog(
    channel="ibm_quantum_platform",
    token=your_api_key,
    instance=your_crn,
)
# You should see a list of Qiskit Functions available to you
# If you encounter the error `QiskitServerlessException: Credentials couldn't be verified`,
# it means your access is not yet active
catalog.list()

In [ ]:
# Load Qedma QESEM function
qesem_function = catalog.load("qedma/qesem")

## Learning Objectives

We'll explore:

1. **Why do we need error mitigation?** - See how quantum noise affects different types of measurements
2. **Kicked Ising circuits** - Our main example is the Kicked Ising chain, a benchmark model in quantum dynamics
3. **Using QESEM** - Use QESEM to mitigate the noise

# Table of Contents

1. [Section 1: Understanding the Importance of Error Mitigation](#Section-1:-Understanding-the-Importance-of-Error-Mitigation)
   - [1.1 Use Case: Kicked Ising](#1.1-Use-Case:-Kicked-Ising)
   - [1.2 Exercise 1: Comparing Ideal and Noisy Values](#1.2-Exercise-1:-Comparing-Ideal-and-Noisy-Values)
     <!-- - [a. Trotter Circuit Visualization](#exercise-1a)
     - [b. Ideal VS Noisy](#exercise-1b)
       - [i. Total Magnetization](#exercise-1b-i)
       - [ii. Heavy Weight (zzz)](#exercise-1b-ii) -->
2. [Section 2: Introduction to QESEM](#Section-2:-Introduction-to-QESEM)
   - [QESEM Function Parameters Reference](#QESEM-Function-Parameters-Reference)
   - [2.1 Exercise 2: Applying QESEM on a simple circuit](#2.1-Exercise-2:-Applying-QESEM-on-a-simple-circuit)
     <!-- - [a. Time Estimation (Analytical + Empirical)](#exercise-2a)
     - [b. Execution](#exercise-2b)
     - [c. Reading Results](#exercise-2c)
     - [d. Execution Metrics](#exercise-2d) -->
   - [2.2 Key Concepts](#2.2-Key-Concepts)
     - [2.2.1 Volume and Active Volume](#2.2.1-Volume-and-Active-Volume)
     - [2.2.2 QESEM Runtime Overhead](#2.2.2-QESEM-Runtime-Overhead)
   - [2.3 Exercise 3: QPU time vs. Active Volume](#2.3-Exercise-3:-QPU-time-vs.-Active-Volume)
     <!-- - [a. Observables vs QPU Time](#exercise-3a)
     - [b. Influence of Angles](#exercise-3b) -->
   - [2.4 Exercise 4: Exploring the $T\propto\frac{1}{\varepsilon^2}$ Relationship](#2.4-Exercise-4:-Exploring-the-$T-\propto-\frac{1}{\varepsilon^2}$-Relationship)
     <!-- - [a. Mitigation Overhead vs 𝜖](#exercise-4a) -->

# Section 1: Understanding the Importance of Error Mitigation

Quantum computers are inherently noisy devices. Even small amounts of noise can significantly affect quantum computations, especially as circuit depth increases. In this section, we'll:

1. **Build a circuit** for the Kicked Ising model
2. **Learn the effects of noise** on different types of quantum observables

## 1.1 Use Case: Kicked Ising

The **Kicked Ising Model** is a paradigmatic quantum spin chain model used to study quantum chaos, entanglement, and non-equilibrium dynamics. It consists of spin-1/2 particles (qubits) with nearest-neighbor interactions on a graph, and a periodically applied "kick".

### Model Definition

The time evolution of the kicked Ising model is governed by a periodically time-dependent Hamiltonian. The unitary evolution over one period $T$, on a 1D lattice of size $n$, is given by:

$$
U = \exp\left(-i J \sum_{j=0}^{n-2} \sigma_j^z \sigma_{j+1}^z \right) \exp\left(-i h_x \sum_{j=0}^{n-1} \sigma_j^x \right)
$$

where $\sigma_j^x, \sigma_j^z$ are Pauli matrices acting on site $j$, and $J, h_x$ are constants.

### Physical Significance

- **Quantum Chaos:** The kicked Ising model exhibits rich quantum chaotic behavior, making it a benchmark for studies of thermalization and information scrambling in many-body quantum systems.
- **Floquet Systems:** Since the system is driven periodically, it is a canonical example of a **Floquet system**, where stroboscopic (periodic) dynamics are studied.
- **Entanglement & Quantum Information:** The model is widely used to investigate entanglement growth, operator spreading, and out-of-time-ordered correlators (OTOCs).

### Applications

- Modeling non-equilibrium quantum dynamics.
- Studying quantum information scrambling and entanglement spreading.
- Exploring the transition between integrability and chaos in quantum systems.

---

**References:**
- Prosen, T. (2007). "Chaos and complexity in a kicked Ising spin chain." *Phys. Rev. E* 65, 036208.
- [Wikipedia: Kicked Ising Model](https://en.wikipedia.org/wiki/Kicked_Ising_model)

<div class="alert alert-block alert-info">
<b>Note:</b>

While in this lab we focus on exploring a 1D Kicked Ising model for simplicity, the QESEM qiskit function can work on arbitrary circuits defined on the heavy-hex lattice connectivity
available on IBM systems.

</div>

## 1.2 Exercise 1: Comparing Ideal and Noisy Values

This exercise demonstrates how quantum noise degrades computation results by comparing three scenarios:

**What We'll Do:**
- **Build Kicked Ising Circuits**: Create quantum circuits simulating time evolution with increasing number of Trotter steps  
- **Measure Observables**:  
  - **Z Correlation Operators**: Evaluate  $\left\langle Z_0 Z_1 \dots Z_{n-1} \right\rangle$ and $\left\langle Z_0 Z_1 \dots Z_{4} \right\rangle$ to probe global multi-qubit correlations  
- **Compare Results**: Ideal expectation values to noisy ones


### Step 1: Preparing Kicked-Ising circuit 

kicked_ising_1D: Creates a quantum circuit that implements the Kicked Ising model. It applies alternating layers of RX gates (transverse field) and RZZ gates (interaction terms) to simulate the time evolution of the system.  

remove_idle_qubits: Removes unused qubits from a quantum circuit

In [ ]:
def kicked_ising_1D(num_qubits: int, theta_x: float, theta_zz: float, s: int) -> QuantumCircuit:
    """
    Parameters:
        num_qubits (int): number of qubits on chain. 
        theta_x (float): Angle for RX gates.
        theta_zz (float): Angle for RZZ gates.
        s (int): Number of steps.
    
    Returns:
        QuantumCircuit: The resulting quantum circuit.
    """
    graph = nx.path_graph(num_qubits)
    qc = QuantumCircuit(num_qubits)

    # Precompute edge layers (alternating non-overlapping pairs)
    edges = list(graph.edges())
    even_edges = [(u, v) for (u, v) in edges if u % 2 == 0]
    odd_edges  = [(u, v) for (u, v) in edges if u % 2 == 1]

    for step in range(s):
        # RX on all qubits
        for q in range(num_qubits):
            qc.rx(theta_x, q)
        
        # Apply even and odd layers separately
        for edge_layer in [even_edges, odd_edges]:
            for u, v in edge_layer:
                qc.rzz(theta_zz, u, v)

        if step < s-1:
            qc.barrier()
    
    return qc

def remove_idle_qubits(qc:QuantumCircuit):
    """
    Removes qubits that are idle throughout the circuit
    """
    used = ({q for op in qc for q in op.qubits})
    dag = circuit_to_dag(qc)
    for q in set (qc.qubits) - used:
        dag.remove_qubits (q)
    return dag_to_circuit(dag)

### Step 2: Kicked Ising Circuit Visualization  

Here we set up the circuit experiment parameters and create a visualization of the circuit.


In [ ]:
n_qubits_ex1 = 20
n_steps = 3  

circ = kicked_ising_1D(n_qubits_ex1, theta_x=np.pi/6, theta_zz=np.pi/3, s=n_steps)

print(f"Circuit 2q layers: {circ.depth(filter_function=lambda instr: len(instr.qubits) == 2)}")     
print(f"\nCircuit structure:")

circ.draw('mpl', scale=0.8, fold=-1)
plt.show()

### Step 3: Simulation Parameters

Here we define: 

1. A list of circuits to simulate (corresponding to the different number of time steps)
2. A list of observables to measure: $\langle Z_0...Z_4 \rangle$ and $\langle Z_0...Z_{n-1} \rangle$ and their labels 


In [ ]:
steps_range = range(1,9)

circs_ex1=[]        
for n_steps in (steps_range):  
    circs_ex1.append(kicked_ising_1D(n_qubits_ex1,  theta_x=np.pi*0.14, theta_zz=np.pi*0.05, s=n_steps))   

# Prepare pairs of  (observables , labels)
observable_label_pairs = [
  (SparsePauliOp.from_sparse_list([("Z"*5, range(5),1)], n_qubits_ex1),  r"$Z_0Z_1...Z_{4}$)"),
  (SparsePauliOp.from_sparse_list([("Z"*n_qubits_ex1, range(n_qubits_ex1),1)], n_qubits_ex1),  r"$Z_0Z_1...Z_{n-1}$")
]

### Step 4: Compute ideal values

Here we compute the ideal expectation values for each observable using exact simulation with statevectors. <br>
This gives us the theoretical values that we would obtain on a perfect quantum computer without any noise.

The following dictionary holds the ideal/noisy expectation values for every observable per step:<br>
graphs["ideal" or "noisy"][observable label] = [Expectation of observable at steps_range[0], Expectation of observable at steps_range[1], ....]

<div class="alert alert-block alert-success">

<b>[Ungraded] Exercise 1.1:</b>

Complete the code for computing the ideal expectation value of `obs` after running the circuit `circ`

Hint: [Statevector Documentation](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.Statevector)

</div>

In [ ]:
graphs={'ideal':{}}
graphs['ideal']['steps_range'] = steps_range
for circ in tqdm(circs_ex1):                       # loop over circuits [one step circuit, two steps circuit ,....]
    for obs, label in observable_label_pairs:  # loop over observables and their label
        if label not in graphs['ideal']:       # create list of ideal expectation values for every circuit
            graphs['ideal'][label]=[]
        # ---- TODO: Exercise 1.1 ---- 
        ideal_value = 
        # ---- End of TODO ----
        graphs['ideal'][label].append(ideal_value)

In [ ]:
# Function to draw graphs from "graphs" object

def graph_plots(graphs):
    """
    Input: graphs object (dictionary)
    Display graphs of noisy (optional) and ideal values vs step 
    """
    fig, axs = plt.subplots(1,2 ,figsize=(15, 4))
    fig.subplots_adjust(wspace=0.5, hspace=0.3)
    
    j=0  # axis index
    for obs, obs_label in observable_label_pairs:
        ax=axs[j]
        ax.set_title("Kicked Ising Model - ideal vs noisy")
        ax.set_ylabel(r"$\langle$"+obs_label+r"$\rangle$")
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))    # Makes x axis integers
        ax.set_xlabel("Steps")
        ax.plot(graphs['ideal']['steps_range'], graphs['ideal'][obs_label], label='Ideal values', marker='.', color='black')
        if 'noisy' in graphs:   # Plot the noisy graph if it exists.
            ax.errorbar(graphs['noisy']['steps_range'], graphs['noisy'][obs_label], np.array(graphs['noisy_std'][obs_label])**0.5, label='Noisy values', marker='.', capsize=3)
        ax.legend()
        ax.grid()
        j=j+1
            
    plt.show()

# Running the function
graph_plots(graphs)

<div class="alert alert-block alert-info">
<b>Build Intuition:</b>
    
Which of the two observables will be more affected by noise?
</div>

### Step 5: Compute noisy values with error bars

Let's add **realistic quantum noise** to our simulation. We'll use an AerSimulator based on IBM's Fake Fez backend, which includes realistic noise models based on actual quantum hardware. <br>

In [ ]:
GPU = 'GPU' in AerSimulator().available_devices()     # Use GPU if available.
fake_backend = FakeFez()
basis_gates = fake_backend.configuration().basis_gates

noisy_backend =  AerSimulator(
    method='matrix_product_state',
    noise_model=NoiseModel.from_backend(fake_backend),
    basis_gates=basis_gates,
    coupling_map=fake_backend.configuration().coupling_map,
    properties=fake_backend.properties(),
    device='GPU'*GPU + 'CPU'*(1-GPU),
)

num_shots = 1000     #### <------ Edit if too slow

Let's compute the noisy expectation values by running the circuits on the noisy simulator with finite shots. We calculate both the expectation values and their standard deviations to understand the statistical uncertainty in our measurements.

<div class="alert alert-block alert-success">
<b>[Ungraded] Exercise 1.2:</b>

Complete the code for running "num_shots" shots of "transpiled_circ" on "noisy_backend" and extract the "counts" dictionary from the result.

Change the number of shots if the simulation is too slow.
    
**Hint:** [AerSimulator Documentation](https://qiskit.github.io/qiskit-aer/tutorials/1_aersimulator.html)
</div>

In [ ]:
graphs['noisy'] = {'steps_range':steps_range}
graphs['noisy_std'] = {}

for circ in tqdm(circs_ex1):       
    transpiled_circ = transpile(circ, basis_gates=basis_gates, coupling_map=fake_backend.coupling_map)    
    transpiled_circ = remove_idle_qubits(transpiled_circ)

    transpiled_circ.measure_all()
    # ---- TODO: Exercise 1.2 ---- 
    counts =  
    # ---- End of TODO ----

    for obs, label in observable_label_pairs:
        if label not in graphs['noisy']:
            graphs['noisy'][label]=[]
            graphs['noisy_std'][label]=[]
    
        noisy_obs_expectation = qiskit.result.sampled_expectation_value(counts,obs)                 # Convert counts to expectation value
        noisy_obs_variance = (1-qiskit.result.sampled_expectation_value(counts,obs)**2)/num_shots   # Only true for Paulis: P^2=I
        graphs['noisy'][label].append(noisy_obs_expectation)
        graphs['noisy_std'][label].append((noisy_obs_variance)**0.5)   

### Step 6: Plot the results

Finally, we visualize the results by plotting both the ideal and noisy expectation values as a function of the number of Trotter steps.

In [ ]:
graph_plots(graphs)

<div class="alert alert-block alert-info">
<b>Build Intuition:</b>
    
- As circuits get deeper, quantum noise accumulates and the gap between ideal and noisy values grows. QESEM allows to run deeper circuits and still get meaningful results.
- The decay of the noisy value is controlled by the number of noisy operations the observable is sensitive to. The $\langle Z0...Z19 \rangle$ observable decays much more than $\langle Z0...Z4 \rangle$ because its lightcone spans the entire circuit.

# Section 2: Introduction to QESEM

Now that we've seen how dramatically noise affects quantum computations, let's explore **QESEM** - a powerful technique to reduce these errors.

### How QESEM Works

QESEM combines two approaches:

1. **Error Suppression**: Reduce the unitary part of the noise. It is based on QESEM's characterization.
2. **Error Mitigation**: Use classical post-processing to estimate what the results would have been without noise.

### QESEM Function Parameters Reference
During this section we will guide you through the main function parameters and explain each one.  
There are more advanced parameters which we won't cover, see an elaborate explanation about all the function parameters [here](https://docs.quantum.ibm.com/guides/qedma-qesem#function-parameters).

## 2.1 Exercise 2: Applying QESEM on a simple circuit

### Step 1: Initialize QESEM Function

In [ ]:
# Load the Qiskit Functions Catalog
# your_api_key = "deleteThisAndPasteYourAPIKeyHere"
# your_crn = "deleteThisAndPasteYourCRNHere"

# catalog = QiskitFunctionsCatalog(
#     channel="ibm_quantum_platform",
#     token=your_api_key,
#     instance=your_crn,
# )

qesem_function = catalog.load("qedma/qesem")

### Step 2: Prepare Circuit and Observables
The following circuit prepares the state $|\psi\rangle = 0.6|0000\rangle+0.8|1111\rangle $

We will measure two observables:<br>
avg_magnetization_ex2 = $\frac{1}{4} \sum_j Z_j$ <br>
all_z_ex2 = $Z_0...Z_3$

In [ ]:
circ_ex2 = qiskit.QuantumCircuit(4)
circ_ex2.ry(0.927*2, 0)   
circ_ex2.cx(0, 1)
circ_ex2.cx(1, 2)
circ_ex2.cx(2, 3)

circ_ex2.draw("mpl", scale=0.7)
plt.show()

avg_magnetization_ex2 = SparsePauliOp.from_sparse_list(
    [("Z", [q], 1 / 4) for q in range(4)], num_qubits=4)

all_z_ex2 = SparsePauliOp.from_sparse_list(
    [("Z"*4, range(4),1)], 4)

obs_list_ex2 =  [avg_magnetization_ex2, all_z_ex2]

### Step 3: Empirical Time Estimation

Users would typically want to know how much QPU time is required for their experiment.
However, this is considered a hard problem for classical computers.<br>
QESEM offers two modes of time estimation to inform users about the feasibility of their experiments:
1. Analytical time estimation - gives a very rough estimation and requires no QPU time. This can be used to test if a transpilation pass would potentially reduce the QPU time. 
2. Empirical time estimation (demonstrated here) - gives a pretty good estimation and uses a few minutes of QPU time.

In both cases, QESEM outputs the time estimation for reaching the required precision for <b>all</b> observables. 

<div class="alert alert-block alert-warning">

**⚠️ Warning: QPU Time Consumption**

Running the cell below may submit a job to a QPU and consume real QPU time. Please ensure you intend to proceed. For learning purposes, you can use a fake QPU.

**Estimated QPU runtime:** 0 minutes 0 seconds (based on tests on `fake_fez`)

</div>

In [ ]:
# Start a job for empirical time estimation

estimation_job_ex2 = qesem_function.run(
    pubs=[(circ_ex2, obs_list_ex2)],
    instance=your_crn,
    backend_name='fake_fez',  # E.g. "ibm_brisbane"
    options={
        "estimate_time_only": "empirical",  # "empirical" - gets actual time estimates without running full mitigation
        "default_precision": 0.01,          # Default precision for all observable. Precision per observable can be defined in the pubs parameter.
        "max_execution_time": 120,          # Limits the QPU time, specified in seconds.
    }
)

In [ ]:
# Get the result object (blocking method). Use job.status() in a loop for non-blocking. 
# This takes a 1-3 minutes
result = estimation_job_ex2.result()   

In [ ]:
print (f"Empirical time estimation (sec): {result[0].metadata['time_estimation_sec']}")

### Step 4: Use QESEM to estimate the expectation values

<div class="alert alert-block alert-success">
<b>Exercise 2:</b>

Use QESEM to estimate the expectation values of `avg_magnetization_ex2` and `all_z_ex2` observables on the state generated by `circ_ex2`.
Set the precision to `0.1`, use `fake_fez` as the backend, and set the maximum QPU time to `15` minutes (`900` seconds).

**Hint:** Remove the options key `estimate_time_only`
</div>

In [ ]:
# choose parameters for the QESEM job:

# job_ex2 = qesem_function.run(
#     pubs=pubs_ex2,
#     instance=instance_ex2,
#     backend_name=backend_name_ex2,  
#     options= options_ex2
# )

# ---- TODO: Exercise 2 ---- 
pubs_ex2 = 
instance_ex2=
backend_name_ex2=
options_ex2={}
# ---- End of TODO ----

### Test parameters

In [ ]:
message = grade_ex2(
    pubs_ex2= pubs_ex2,
    backend_name_ex2=backend_name_ex2,
    options_ex2=options_ex2
)
print (message)

### Start a job only if parameter check passed

<div class="alert alert-block alert-warning">

**⚠️ Warning: QPU Time Consumption**

Running the cell below may submit a job to a QPU and consume real QPU time. Please ensure you intend to proceed. For learning purposes, you can use a fake QPU.

**Estimated QPU runtime:** 0 minutes 0 seconds (based on tests on `fake_fez`)

</div>

In [ ]:
if message == correct_message:     # Run the job only if parameters are correct
    # Start a job
    job_ex2 = qesem_function.run(
        pubs=pubs_ex2,
        instance=instance_ex2,
        backend_name=backend_name_ex2,  
        options= options_ex2
    )
    print ("Job sent.")
else:
    print ("Parameter check failed, job did not start.")

### Step 5: Reading the results

In [ ]:
result = job_ex2.result()  # Blocking - takes 3-5 minutes
noisy_results = result[0].metadata["noisy_results"]
for en,obs in enumerate(obs_list_ex2):
    print ("-"*10)
    print ("Observable: "+['Average Magnetization','ZZZZ'][en])
    print (f"Ideal: {Statevector(circ_ex2).expectation_value(obs).real}")
    print (f"Noisy: {noisy_results.evs[en]} \u00B1 {noisy_results.stds[en]}")
    print (f"QESEM: {result[0].data.evs[en]} \u00B1 {result[0].data.stds[en]}")
    
print ("-"*10)
print (f"Gate fidelities found: {result[0].metadata['gate_fidelities']}")    # Some of the data gathered during a QESEM run.

In [ ]:
# Results as a graph
x = np.arange(2)  # Column position
width = 0.06       

fig, ax = plt.subplots(figsize=(4,2.5*1.5))

# Plot the bars side by side
ax.bar(x - width,   [Statevector(circ_ex2).expectation_value(obs).real for obs in [avg_magnetization_ex2, all_z_ex2]], width, label='Ideal')
ax.bar(x,           noisy_results.evs, width, label='Noisy',  yerr=noisy_results.stds,  capsize=6)
ax.bar(x + width,   result[0].data.evs,   width, label='QESEM', yerr=result[0].data.stds,  capsize=6)

ax.set_xticks(x)
ax.set_xticklabels(['Average Magnetization','ZZZZ'])
ax.set_title(r"Comparing Ideal, Noisy and QESEM for $|\psi\rangle = 0.6|0000\rangle+0.8|1111\rangle$ ")
ax.legend()

plt.tight_layout()
plt.grid(axis='y')
plt.axhline(0, color='black', linewidth=2.5)  # y=0, bold black line
plt.show()

## 2.2 Key Concepts

### 2.2.1 Volume and Active Volume

A key figure of merit for quantifying the hardness of both error mitigation and classical simulation for a given circuit and observable is **active volume**: The number of two-qubit gates affecting the observable in the circuit. Different circuits and observables have different **active volumes**, which affects how hard they are to error-mitigate.

The active volume depends on:
- Circuit depth and width
- Observable weight (number of non-identity Pauli operators)
- Circuit structure (light cone of the observable)

For further details, see the talk from the [2024 IBM Quantum Summit](https://www.youtube.com/watch?v=Hd-IGvuARfE&t=1730s&ab_channel=IBMResearch).

<div>
<img src="active_vol.svg" />
</div>

### 2.2.2 QESEM Runtime Overhead

The QPU time behaves roughly like:
$$T_{QPU} \approx \left(\frac{A}{\epsilon^2}\right) \times e^{C \times IF \times V_{active}} + B$$  
- B overhead of gate optimization and error characterization
- precision $\epsilon$ absolute error in expectation value
- $ IF \times V_{active} $ infidelity-per-gate times active volume. The active volume only includes gates within the active light-cone.

The **overhead** of error mitigation tells us how many additional quantum resources we need to achieve a target precision.

## 2.3 Exercise 3: QPU time vs. Active Volume
In this section we'll fix the number of steps and the precision, and compare the QPU time for the two observables in Exercise 1. 


<div class="alert alert-block alert-success">
<b>Exercise 3:</b>

Use QESEM's **analytical** time estimation for measuring the expectation of $\langle Z_0...Z_4\rangle$ and $\langle Z_0...Z_{n-1}\rangle$ of the state generated by a few steps of the Kicked Ising model.

Set the `default_precision` to `0.02`.

</div>

In [ ]:
# Set the options parameter for the QESEM job by the instruction:

# ---- TODO: Exercise 3 ---- 
options_ex3={
    "estimate_time_only": ,
    "default_precision": ,
}
# ---- End of TODO ----

### Test parameters

In [ ]:
message = grade_ex3(options_ex3)
print (message)

### Start a jobs only if parameter check passed

<div class="alert alert-block alert-warning">

**⚠️ Warning: QPU Time Consumption**

Running the cell below may submit a job to a QPU and consume real QPU time. Please ensure you intend to proceed. For learning purposes, you can use a fake QPU.

**Estimated QPU runtime:** 0 minutes 0 seconds (based on tests on `fake_fez`)

</div>

In [ ]:
if message == correct_message:
    print("Starting jobs")
    selected_step = 6   # number of steps in the circuit. 
    num_qubits_ex3 = 20
    
    
    circ = kicked_ising_1D(num_qubits_ex3, theta_x=np.pi*0.14, theta_zz=np.pi*0.05, s=selected_step)
    
    # Prepare pairs of  (observables , labels)
    observable_label_pairs_ex3 = [
        (SparsePauliOp.from_sparse_list([("Z"*5, range(5),1)], num_qubits_ex3),  r"$Z_0Z_1...Z_{4}$"),
        (SparsePauliOp.from_sparse_list([("Z"*num_qubits_ex3, range(num_qubits_ex3),1)], num_qubits_ex3),  r"$Z_0Z_1...Z_{n-1}$")
    ] 
    
    volume_jobs = []
    for observable, obs_label in observable_label_pairs_ex3:
        # run analytical time estimation on each observable separately
        job = qesem_function.run(
            pubs=[(circ, [observable])],
            instance=your_crn,
            backend_name="fake_fez",  
            options=options_ex3
        )
        volume_jobs.append(job)
else:
    print ("Parameters check failed. Jobs did not start")

In [ ]:
# Print the analytical time estimation. Takes about 3 minutes
job_index=0
for observable, obs_label in observable_label_pairs_ex3:
    # Get the result and extract time estimation
    result = volume_jobs[job_index].result()
    job_index+=1
    print (f"Analytical time estimation for observable {obs_label}: {result[0].metadata['time_estimation_sec']/60} min")

<div class="alert alert-block alert-info">
<b>Build Intuition:</b> 

While the analytical time estimation is very rough (resolution of 30 minutes) and pessimistic, it still captures the difference in the active volumes of the two observables fairly quickly, and without using any QPU time.
</div>

## 2.4 Exercise 4: Exploring the $T \propto \frac{1}{\varepsilon^2}$ Relationship

In this exercise, we will investigate **how QPU time scales with precision requirements**.
### Step 1: Create and visualize the test circuit

We'll create a single Kicked Ising circuit to test the precision-time relationship. This circuit will have enough complexity to show clear scaling behavior while remaining manageable for the experiment.

<div class="alert alert-block alert-success">
<b>Exercise 4.1:</b>

Complete the code. Use the function `kicked_ising_1D()` defined in Exercise 1 to create the circuit `circ_ex4`. Be sure to use the variables defined below `n_qubits_ex4`, `n_steps_ex4`, `theta_x_ex4`, `theta_zz_ex4`.
</div>

In [ ]:
n_qubits_ex4 = 5           # Number of qubits (enough to see scaling effects)
n_steps_ex4 = 10           # Number of Trotter steps (creates sufficient circuit depth)
theta_x_ex4 = np.pi/6      # Rotation angle for X gates (transverse field strength)
theta_zz_ex4 = np.pi/3     # Rotation angle for ZZ gates (interaction strength)

# Create the Kicked Ising circuit that will be used for precision testing
# ---- TODO: Exercise 4.1 ---- 
circ_ex4 = 
# ---- End of TODO ----

In [ ]:
message = grade_ex_4_1(circ_ex4)
print (message)

In [ ]:
print(f"Circuit 2q layers: {circ_ex4.depth(filter_function=lambda instr: len(instr.qubits) == 2)}")     
print(f"\nCircuit structure:")

circuit_drawer = circ_ex4.draw('mpl', scale=1, fold=-1)
plt.show()

### Step 2: Submit time estimation jobs with different precision values

We'll submit multiple QESEM jobs, each with a different precision requirement (ε). The range from 0.005 to 0.06 covers both high-precision (expensive) and moderate-precision (cheaper) regimes.

The observable: $\langle Z_0...Z_4 \rangle$. <br>


**Empirical time estimation**: We will use "empirical" time estimation this setting performs a small execution on the QPU without running full mitigation. This usually takes a few minutes per estimation and is more reliable than the "analytical" estimation.

In [ ]:
backend_name_fake = "fake_fez"
precisions_ex4 = [0.005, 0.01, 0.03, 0.05] 
observable_ex4 = qiskit.quantum_info.SparsePauliOp.from_sparse_list([("Z"*n_qubits_ex4, range(n_qubits_ex4),1)], n_qubits_ex4) # Z_0..Z_{n-1}

# Initialize list to store job IDs
jobs_ex4 = []

<div class="alert alert-block alert-warning">

**⚠️ Warning: QPU Time Consumption**

Running the cell below may submit a job to a QPU and consume real QPU time. Please ensure you intend to proceed. For learning purposes, you can use a fake QPU.

**Estimated QPU runtime:** 0 minutes 0 seconds (based on tests on `fake_fez`)

</div>

In [ ]:
if message == correct_message:
    # Loop through each precision value
    for precision in precisions_ex4:
        print(f"Submitting job with precision: {precision:.3f}")
        
        # Start a job with current precision
        job = qesem_function.run(
            pubs=[(circ_ex4, [observable_ex4], [], precision)],
            instance=your_crn,
            backend_name=backend_name_fake,  # E.g. "ibm_brisbane"
            options={
                "estimate_time_only": "empirical",  # "empirical" - gets actual time estimates without running full mitigation
                "max_execution_time": 240,          # limits the QPU time, specified in seconds.
            }
        )
        
        # Store the job ID
        jobs_ex4.append(job)
        print(f"Job submitted with ID: {job.job_id}")
    
    print(f"\nAll jobs submitted!")
    
    start_time = time.time()
else:
    print ("Parameter check failed. Jobs did not start")

### Step 3: Monitor job progress

Since time estimation jobs can take several minutes to complete, we'll monitor their progress. This gives us real-time feedback on when all jobs are finished. Might take approximately 14 minutes (tested on fake_fez).

In [ ]:
print("Monitoring job completion...   (this takes around 14 minutes, tested on fake_fez, you may break and return later)")
print(f"Total jobs: {len(jobs_ex4)}")
print("-" * 30)

while True:  # Each job performs device characterization to estimate QPU time requirements
    # Count how many jobs are done
    done_count = 0
    for job in jobs_ex4:  # Jobs with smaller ε values may take longer to process
        if job.status() == "DONE":
            done_count += 1
    
    # Show current status
    elapsed_time = time.time() - start_time
    timestamp = datetime.now().strftime('%H:%M:%S')
    print(f"[{timestamp}] {done_count}/{len(jobs_ex4)} jobs completed (elapsed: {elapsed_time/60:.1f} min)")
    
    # Check if all jobs are done
    if done_count == len(jobs_ex4):
        print(f"\nAll jobs completed!")
        break
    
    # Wait 1 minute before next check
    time.sleep(60)

end_time = time.time()
print(f"Total time: {(end_time - start_time)/60:.1f} minutes")

### Step 4: Extract time estimation results

Once all jobs are complete, we extract the QPU time estimates from each job's metadata. These time estimates represent how long QESEM would need to achieve the requested precision.

In [ ]:
# Once jobs are complete, extract time estimations
time_estimations = []

print("\nExtracting time estimations from jobs...")
for i, job in enumerate(jobs_ex4):  # Loop through all completed jobs
    print(f"Getting time estimation for job {i+1}")
    
    # Get the result and extract time estimation
    time_estimate_result = job.result()
    time_estimation_sec = time_estimate_result[0].metadata['time_estimation_sec']
    
    # Store the time estimation
    time_estimations.append(time_estimation_sec)
    print(f"Time estimation: {time_estimation_sec} seconds")

print(f"\nAll time estimations extracted!")


### Step 5: Visualize the results

We'll create a standard linear plot to visualize how QPU time varies with precision. This gives us an intuitive view of the relationship.

**Expected Pattern**: Time estimates should increase dramatically as precision requirements become more stringent (smaller ε).

In [ ]:
# Define the plotting function to visualize QPU time vs precision
def plot_qpu_time_vs_precision(precisions, time_estimations_sec, title_suffix=""):
    """
    Plot QPU time estimation vs precision with curve fitting
    
    Args:
        precisions: Array of precision values (ε)
        time_estimations_sec: Array of time estimations in seconds
        title_suffix: Optional suffix to add to the plot title
    """
    # Convert time estimations from seconds to minutes
    time_estimations_min = np.array(time_estimations_sec) / 60

    # Define the fitting function: T = A/precision^2 + B
    def fit_function(precision, A, B):
        return A / (precision**2) + B

    # Perform the curve fit using time
    popt, pcov = curve_fit(fit_function, precisions, time_estimations_min)
    A_fit, B_fit = popt

    # Create a smooth curve for plotting the fit
    precision_smooth = np.linspace(min(precisions), max(precisions), 100)
    time_fit = fit_function(precision_smooth, A_fit, B_fit)

    plt.figure(figsize=(8, 5))
    plt.plot(precisions, time_estimations_min, 'o', markersize=8, color='blue', label='Data')
    plt.plot(precision_smooth, time_fit, '-', linewidth=2, color='red', label=f'Fit: T = {A_fit:.3f}/ε² + {B_fit:.1f}')
    plt.xlabel('Precision (ε)', fontsize=12)
    plt.ylabel('QPU Time Estimation (minutes)', fontsize=12)
    plt.title(f'QESEM QPU Time Estimation vs Precision{title_suffix}', fontsize=14)

    plt.locator_params(axis='y', nbins=10)  # Increase number of y-axis ticks
    plt.grid(True, alpha=0.3)
    plt.legend()

    # Display fit parameters
    print(f"Fitted parameters:")
    print(f"A = {A_fit:.3f}")
    print(f"B = {B_fit:.2f}")
    print(f"Function: T = {A_fit:.3f}/ε² + {B_fit:.2f}")
    print(f"Data points: {len(precisions)}")

    plt.tight_layout()
    plt.show()

# Create the initial plot with Steps 2-5 data
plot_qpu_time_vs_precision(precisions_ex4, time_estimations)

And indeed we can see the expected functional dependence of:
$$T_{QPU} \approx \frac{\tilde{A}}{\epsilon^2} + B$$
Where:
- $\tilde A={A} \times e^{C \times IF \times V_{active}}$
- $A$ is a constant that depends on the circuit and the expectation value
- $B$ is overhead due to gate optimization and error characterization
- $C$ is a constant between 2-4 that depends on the details of the error mitigation method. 

### Step 6: Execute with error mitigation on a real hardware

Now let's run the actual QESEM error mitigation on our circuit.

<div class="alert alert-block alert-success">
<b> Exercise 4.2:</b>

Look at the results of the QPU time vs precision graph above and think which precision you would like for your execution on real quantum hardware.
Complete the QESEM job parameters accordingly.

We will run an empirical time estimation for your desired precision to predict QPU resource requirements before running the full QESEM mitigation. This will take less than 5 minutes QPU time.
    
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> Use a moderate precision for reasonable execution time while still demonstrating the mitigation capabilities.
</div>

<div class="alert alert-block alert-warning">

**⚠️ Warning: QPU Time Consumption**

This exercise should use use a real QPU (e.g. `ibm_fez). Please ensure you intend to proceed. 

**Estimated QPU runtime:** 1 minutes 45 seconds (based on tests on `ibm_fez`)

</div>

In [ ]:
# ---- TODO: Exercise 4.2 ---- 
precision_ex4 =
pubs_ex4 = [( , , [], precision_ex4)]  # Complete the pubs
instance_ex4 =
backend_name_ex4 =
options_ex4 = {
    "estimate_time_only": ,
    "max_execution_time": ,
    "execution_mode":  # "batch" or "session" for less QPU time or faster execution
}
# ---- End of TODO ----

### Test parameters

In [ ]:
message = grade_ex_4_2(
    precision_ex4= precision_ex4,
    pubs_ex4= pubs_ex4,
    backend_name_ex4=backend_name_ex4,
    options_ex4=options_ex4
)
print(message)

### Start a job only if parameter check passed

In [ ]:
if message == correct_message:     # Run the job only if parameters are correct
    # Start a job
    # Get time estimation first (quick empirical check)
    time_est_job = qesem_function.run(
        pubs = pubs_ex4,
        instance = instance_ex4,
        backend_name = backend_name_ex4,
        options = options_ex4
    )

    print(f"Time estimation job submitted: {time_est_job.job_id}")
    print(f"with precision: {precision_ex4}")
    
else:
    print (message)
    print ("Parameter check failed. Job did not start.")

<a id="tips"></a>
<div class="alert alert-block alert-info">
    
<b> Tip:</b> Use this code cell to check your job's status.

In [ ]:
time_est_job.status()

Extract the time estimation from Step 6a and add it to our precision vs. QPU time graph to see how the new data point fits the T ∝ 1/ε² relationship.

In [ ]:
# Get the time estimation result
time_est_result = time_est_job.result()
step6_time_estimation = time_est_result[0].metadata['time_estimation_sec']

print(f"Step 6 time estimation: {step6_time_estimation} seconds")

# Add the new data point to existing arrays
precisions_updated = np.append(precisions_ex4, precision_ex4)
time_estimations_updated = np.append(time_estimations, step6_time_estimation)

# Plot the updated graph with the new data point
print("\nUpdated graph with Step 6 data point:")
plot_qpu_time_vs_precision(precisions_updated, time_estimations_updated, " (Including Step 6)")

<div class="alert alert-block alert-success">
    
<b> Exercise 4.3:</b> 

If you are satisfied with the time estimation go ahead and run the full QESEM error mitigation on our 5-qubit, 10-step Kicked Ising circuit with precision ε. This will take QPU time according to your estimation result.
Complete the QESEM job parameters accordingly.   
    
</div>

<a id="tips"></a>
<div class="alert alert-block alert-info">
    
<b> Tip:</b> Use `max_execution_time` option, this allows you to limit the QPU time, specified in seconds. After the time limit is reached, QESEM stops sending new circuits. Circuits that have already been sent continue running, you can see detailed explanation [here](https://docs.quantum.ibm.com/guides/qedma-qesem#function-parameters).

Note: QESEM will end its run when it reaches the target precision or when it reaches `max_execution_time`, whichever comes first.

In [ ]:
precision_ex4_miti = precision_ex4
pubs_ex4_miti = pubs_ex4
instance_ex4_miti = instance_ex4
backend_name_ex4_miti = backend_name_ex4

# ---- TODO: Exercise 4.3 ---- 
options_ex4_miti = {
    "max_execution_time": ,  # In seconds
    "execution_mode":        # "batch" or "session" for less QPU time or faster execution
}
# ---- End of TODO ----

### Test parameters

In [ ]:
message = grade_ex_4_3(
    precision_ex4= precision_ex4_miti,
    pubs_ex4= pubs_ex4_miti,
    backend_name_ex4=backend_name_ex4_miti,
    options_ex4=options_ex4_miti
)
print(message)

### Start a job only if parameter check passed

<div class="alert alert-block alert-warning">

**⚠️ Warning: QPU Time Consumption**

Running the cell below may submit a job to a QPU and consume real QPU time. Please ensure you intend to proceed.

**Estimated QPU runtime:** 3 minutes 0 seconds (based on tests on `ibm_fez`)

</div>

<div class="alert alert-block alert-warning">
<b>Warning:</b> The QPU time estimation changes from one backend to another. Therefore, when executing the QESEM function, make sure to run it on the same backend that was selected when obtaining the QPU time estimation.

In [ ]:
if message == correct_message:     # Run the job only if parameters are correct
    # Start a job
    # Submit actual mitigation job
    mitigation_job = qesem_function.run(
        pubs=pubs_ex4_miti,
        instance=instance_ex4_miti,
        backend_name=backend_name_ex4_miti,
        options=options_ex4_miti
    )
    print(f"QESEM mitigation job submitted: {mitigation_job.job_id}")
    print("Job is running... This may take several minutes.")
    
else:
    print (message)
    print ("Parameter check failed, job did not start.")

<a id="tips"></a>
<div class="alert alert-block alert-info">
    
<b> Tip:</b> Use this code cell to check your job's status.

</div>

In [ ]:
mitigation_job.status()

Extract and analyze the QESEM results, comparing ideal, noisy, and error-mitigated expectation values to demonstrate the effectiveness of QESEM.


In [ ]:
# Read and analyze results
print("Reading QESEM results...")

# Get the mitigated results
qesem_result = mitigation_job.result()
print(f"Job completed successfully!")

# Extract results for our observable
pub_result = qesem_result[0]
mitigated_expectation = pub_result.data.evs[0] 
mitigated_std = pub_result.data.stds[0]

# Get noisy results for comparison
noisy_results = pub_result.metadata["noisy_results"]
noisy_expectation = noisy_results.evs[0]
noisy_std = noisy_results.stds[0]

# Calculate ideal value for comparison
ideal_expectation = Statevector(circ_ex4).expectation_value(observable_ex4).real

# Display comprehensive results summary
print("\n" + "="*60)
print("EXERCISE 4 - QESEM RESULTS SUMMARY")
print("="*60)
print(f"Observable: Global Z measurement (Z^⊗{n_qubits_ex4})")
print(f"Circuit: {n_steps_ex4}-step Kicked Ising, {n_qubits_ex4} qubits")
print(f"Precision target: {precision_ex4}")
print("-"*60)
print(f"Ideal value:      {ideal_expectation:.6f}")
print(f"Noisy value:      {noisy_expectation:.6f} ± {noisy_std:.6f}")
print(f"QESEM value:      {mitigated_expectation:.6f} ± {mitigated_std:.6f}")
print("-"*60)
print(f"Noisy error:      {abs(noisy_expectation - ideal_expectation):.6f}")
print(f"QESEM error:      {abs(mitigated_expectation - ideal_expectation):.6f}")
print(f"Error reduction:  {abs(noisy_expectation - ideal_expectation)/abs(mitigated_expectation - ideal_expectation if mitigated_expectation != ideal_expectation else 1):.1f}x")
print("-"*60)
print(f"QESEM within target precision: {'✓' if abs(mitigated_expectation - ideal_expectation) <= precision_ex4 else '✗'}")
print("-"*60)
# Additional detailed metrics
print(f"Total QPU time: \n {pub_result.metadata['total_qpu_time']:.6f}")
print(f"Gates fidelity measured during the experiment: \n {pub_result.metadata['gate_fidelities']}")
print(f"Total shots / mitigation shots: \n {pub_result.metadata['total_shots']} / {pub_result.metadata['mitigation_shots']}")
print("="*60)

# Pretzel Example (optional)
We've added a 28 circuit for users who would like to experiment with QESEM on higher volume circuits. <br>

<div class="alert alert-block alert-warning">
<b>Warning:</b> Please limit the QPU time according to your resources.
</div>

In [ ]:
# Loading a 6 steps of Kicked Ising Model on a pretzel 
pretzel_qc = QuantumCircuit.from_qasm_file("pretzel6.qasm")    
# Defining observable
avg_magnetization = qiskit.quantum_info.SparsePauliOp.from_sparse_list(
    [("Z", [q], 1 / 28) for q in range(28)], 28
)

# Feedback Survey

We’d love to hear about your experience using the Qiskit Function! Your feedback is valuable and will help Qiskit Function providers enhance their tools and services. Please take a moment to share your thoughts by completing our short 2 min [feedback survey](https://airtable.com/app6VujlNUHZuOnAF/pagpw6TgP9UEt4TAT/form).

## Environment Check: Package Versions

Let's verify that all required packages are installed and check their versions:

In [ ]:
import qiskit
import qiskit_ibm_runtime
import qiskit_ibm_catalog
import qiskit_aer
import matplotlib
import networkx
import tqdm
import numpy
import scipy
import pylatexenc
import IPython

print("Main Qiskit Packages:")
print(f"Qiskit: {qiskit.__version__}")
print(f"Qiskit IBM Runtime: {qiskit_ibm_runtime.__version__}")
print(f"Qiskit IBM Catalog: {qiskit_ibm_catalog.__version__}")
print(f"Qiskit Aer: {qiskit_aer.__version__}")

print("\nScientific:")
print(f"NumPy: {numpy.__version__}")
print(f"SciPy: {scipy.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")

print("\nUtilities:")
print(f"NetworkX: {networkx.__version__}")
print(f"tqdm: {tqdm.__version__}")
print(f"pylatexenc: {pylatexenc.__version__}")
print(f"IPython/Jupyter: {IPython.__version__}")

# Additional information

**Created by:** Yosi Atia, Tali Shnaider

**Advised by:** Ori Alberton, Eyal Bairey, Asaf Berkovitch, Assaf Zubida

**Version:** 1.0.0